# Tensorflow - Loading data - Adjusting the layer to our needs

https://www.kaggle.com/code/roberthatch/gislr-feature-data-on-the-shoulders/notebook

This notebook generates a tensorflow preprocessing layer that transforms the whole dataset of single parquet files into numpy arrays.

The flow is as follow:

***preprocessing layer:***
1. drop z coordinate if desired (this would reduce number of coordinates to 2; following array shapes show number for 3 coordinates)
2. Convert input into x containing avg face, lips, upper pose landmarks, left hand, right hand for every frame, 
    e.g. [23, 106, 3] = (number of frames, landmarks, coordinates)
3. Pad x or cut x such that the length is as defined (e.g. length = 30 --> [30, 106, 3])  = (number of frames, landmarks, coordinates)
4. interpolate single missing values, then replace NaN values with zero
4. Resize it to either flattened or 3 dimensional array:
    * 3D: ( [1, 30, 318] = (1 row, number of frames, landmarks * coordinates) 
    * or flattened: [1, 9540])  = (1 row, number of frames * landmarks * coordinates)

***looping through csv file ***

By looping through the csv file each parquet file will be loaded and the required data will be extracted and transformed by running it through the preprocessing layer.
Then the data will be added to our final numpy array of all recordings. If the recording is exceeding defined length limits it will be removed from the dataset.

* final shape of data: flattened: x (94477, 5796) and y (94477,) or unflattened: (94477, 30, 212) (94477,)
* shape also depends on selected landmarks and coordinates and migth vary

## Import libraries

In [147]:
%pip install tqdm
import os

import json
from tqdm import tqdm
import numpy as np
import pandas as pd
import random

import tensorflow as tf


[notice] A new release of pip is available: 23.1 -> 23.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


## Setup

In [148]:
#file paths for Kaggle
# LANDMARK_FILES_DIR = "/kaggle/input/asl-signs/train_landmark_files"
# TRAIN_FILE = "/kaggle/input/asl-signs/train.csv"
# OUTPUT = ""
# label_map = json.load(open("/kaggle/input/asl-signs/sign_to_prediction_index_map.json", "r"))

#for local notebook, adjust file paths here if required
LANDMARK_FILES_DIR = "../data/asl-signs/"
TRAIN_FILE = "../data/asl-signs/train.csv"
OUTPUT = "../data/" #path to save x and y files
label_map = json.load(open("../data/asl-signs/sign_to_prediction_index_map.json", "r"))

In [149]:
#getting signs for specific label numbers
#converting strings in list to numbers
listsigns = ['31.0', '194.0', '165.0', '234.0', '90.0', '195.0', '239.0', '34.0', '143.0', '182.0', '221.0', '173.0', '55.0', '50.0', '82.0', '52.0', '122.0', '228.0', '155.0', '103.0']
listsigns = [float(x) for x in listsigns]
#get signs from dictionary
signs = []
for key, value in label_map.items():
    if value in listsigns:
        signs.append(key)

print(signs)

['brown', 'callonphone', 'cow', 'cry', 'dad', 'fireman', 'frog', 'gum', 'icecream', 'minemy', 'nose', 'owl', 'please', 'radio', 'shhh', 'shirt', 'tomorrow', 'uncle', 'water', 'who']


## Configuration

In [150]:
#limit dataset for quick test
QUICK_TEST = False
QUICK_LIMIT = 500

#Define length of sequences for padding or cutting; 22 is the median length of all sequences
LENGTH = 22

#define min or max length of sequences; sequences too long/too short will be dropped
#max value of 92 was defined by calculating the interquartile range
MIN_LENGTH = 2
MAX_LENGTH = 120

#final data will be flattened, if false data will be 3 dimensional
FLATTEN = False

#define initialization of numpy array 
ARRAY = False #(True=Zeros, False=empty values)

#Define padding mode 
#1 = padding at start&end; 2 = padding at end; 3 = no padding, 4 = copy first/lastframe, 5 = copy last frame)
#Note: Mode 3 will give you an error due to different lengths, working on that
PADDING = 2
CONSTANT_VALUE = 0 #only required for mode 1 and 2; enter tf.constant(float('nan')) for NaN

#define if z coordinate will be dropped
DROP_Z = False

#mirror, flips x coordinate for data augmentation
MIRROR = True

#define if csv file should be filtered
CSV_FILTER  = False
#define how many participants for test set
TEST_COUNT = 5 #5 participants account for ca 23% of dataset
#generate test or train dataset (True = Train dataset; False = Test dataset)
#TRAIN = True #only works if CSV_FILTER is activated
TRAIN = True

#filter for specific signs
SIGN_FILTER = True
sign_list = [31.0, 194.0, 165.0, 234.0, 90.0, 195.0, 239.0, 34.0, 143.0, 182.0, 221.0, 173.0, 55.0, 50.0, 82.0, 52.0, 122.0, 228.0, 155.0, 103.0]
#Best 20 sings are: [31.0, 194.0, 165.0, 234.0, 90.0, 195.0, 239.0, 34.0, 143.0, 182.0, 221.0, 173.0, 55.0, 50.0, 82.0, 52.0, 122.0, 228.0, 155.0, 103.0]

#define filenames for x and y:
feature_data = 'X_20_fz' #x data
feature_labels = 'y_20_fz' #y data

#use for test dataset
#feature_data = 'X_test_h6' #x data
#feature_labels = 'y_test_h6' #y data


RANDOM_STATE = 42

#Defining Landmarks
#index ranges for each landmark type
#dont change these landmarks
FACE = list(range(0, 468))
FACE_OUTLINE = [  0,   1,   4,   5,  10,  11,  12,  13,  14,  15,  16,  17,  19,
        21,  34,  37,  38,  39,  44,  45,  46,  51,  52,  53,  54,  58,
        61,  62,  63,  65,  66,  67,  68,  69,  70,  71,  72,  76,  78,
        82,  84,  85,  86,  87,  93, 103, 104, 105, 107, 108, 109, 116,
       123, 125, 127, 132, 135, 136, 137, 138, 139, 140, 143, 147, 148,
       149, 150, 151, 152, 156, 162, 169, 170, 171, 172, 175, 176, 177,
       180, 181, 192, 199, 208, 213, 215, 220, 227, 234, 237, 251, 264,
       267, 268, 269, 274, 275, 276, 281, 282, 283, 284, 288, 293, 295,
       296, 297, 298, 299, 300, 301, 302, 314, 315, 316, 317, 323, 332,
       333, 334, 336, 337, 338, 356, 361, 364, 365, 366, 367, 368, 369,
       377, 378, 379, 383, 389, 394, 395, 396, 397, 400, 401, 405, 428,
       435, 440, 444, 445, 447, 454, 457]
LEFT_HAND = list(range(468, 489))
POSE = list(range(489, 522))
POSE_UPPER = list(range(489, 510))
RIGHT_HAND = list(range(522, 543))
LIPS = [61, 185, 40, 39, 37,  0, 267, 269, 270, 409,
                 291,146, 91,181, 84, 17, 314, 405, 321, 375, 
                 78, 191, 80, 81, 82, 13, 312, 311, 310, 415, 
                 95, 88, 178, 87, 14,317, 402, 318, 324, 308]
lipsUpperOuter= [61, 185, 40, 39, 37, 0, 267, 269, 270, 409, 291]
lipsLowerOuter= [146, 91, 181, 84, 17, 314, 405, 321, 375, 291]
lipsUpperInner= [78, 191, 80, 81, 82, 13, 312, 311, 310, 415, 308]
lipsLowerInner= [78, 95, 88, 178, 87, 14, 317, 402, 318, 324, 308]
#defining landmarks that will be merged
averaging_sets = []

#generating list with all landmarks selected for preprocessing
#change landmarks you want to use here:
point_landmarks_right = RIGHT_HAND + lipsUpperInner + lipsLowerInner
point_landmarks_left = LEFT_HAND + lipsUpperInner + lipsLowerInner

#calculating sum of total landmarks used
LANDMARKS = len(point_landmarks_right) + len(averaging_sets)
print(f'Total count of used landmarks: {LANDMARKS}')

#defining input shape for model
if DROP_Z:
    INPUT_SHAPE = (LENGTH,LANDMARKS*2)
else:
    INPUT_SHAPE = (LENGTH,LANDMARKS*3)
print(INPUT_SHAPE)

Total count of used landmarks: 43
(22, 129)


### Helper Functions

In [151]:
ROWS_PER_FRAME = 543
def load_relevant_data_subset(pq_path):
    #defines which columns will be read from the file
    data_columns = ['x', 'y', 'z']
    data = pd.read_parquet(pq_path, columns=data_columns)
    #calculates the number of frames in the data by dividing the length of the data by the number of rows per frame
    n_frames = int(len(data) / ROWS_PER_FRAME)
    #reshapes the data into a 3D array with shape (n_frames, ROWS_PER_FRAME, len(data_columns))
    data = data.values.reshape(n_frames, ROWS_PER_FRAME, len(data_columns))
    return data.astype(np.float32)

In [152]:
def right_hand_percentage(x):
    #calculates percentage of right hand usage
    right = tf.gather(x, RIGHT_HAND, axis=1)
    left = tf.gather(x, LEFT_HAND, axis=1)
    right_count = tf.reduce_sum(tf.where(tf.math.is_nan(right), tf.zeros_like(right), tf.ones_like(right)))
    left_count = tf.reduce_sum(tf.where(tf.math.is_nan(left), tf.zeros_like(left), tf.ones_like(left)))
    return right_count / (left_count+right_count)

def calc_finger_angles(x, joint_list):
    #initialization of empty np array
    x_out = np.zeros((len(x),len(joint_list),1))
    #looping through list of landmarks required for one finger
    for i in range(len(joint_list)):
        #collecting all coordinates for one finger for all frames
        x_joint = tf.gather(x, joint_list[i], axis=1)
        #Loop through joint sets 
        for frame in range(len(x)):
                a = np.array([x_joint[frame][0][0], x_joint[frame][0][1]]) # First coord
                b = np.array([x_joint[frame][1][0], x_joint[frame][1][1]]) # Second coord
                c = np.array([x_joint[frame][2][0], x_joint[frame][2][1]]) # Third coord

                radians = np.arctan2(c[1] - b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
                angle = np.abs(radians*180.0/np.pi)
                    
                if angle > 180.0:
                    angle = 360-angle
                
                x_out[frame][i]= angle

    return x_out               

In [153]:
def tf_nan_mean(x, axis=0):
    #calculates the mean of a TensorFlow tensor x along a specified axis while ignoring any NaN values in the tensor.
    return tf.reduce_sum(tf.where(tf.math.is_nan(x), tf.zeros_like(x), x), axis=axis) / tf.reduce_sum(tf.where(tf.math.is_nan(x), tf.zeros_like(x), tf.ones_like(x)), axis=axis)

def tf_nan_std(x, axis=0):
    #calculates the standard deviation of a tensor x along a specified axis, while ignoring any NaN values in the tensor
    d = x - tf_nan_mean(x, axis=axis)
    return tf.math.sqrt(tf_nan_mean(d * d, axis=axis))

#this function is only required if mean and std will be calculated for specific segments of the data
def flatten_means_and_stds(x, axis=0):
    #Get means and stds
    x_mean = tf_nan_mean(x, axis=0)
    x_std  = tf_nan_std(x,  axis=0)
    #concats mean and std values for each sequence
    x_out = tf.concat([x_mean, x_std], axis=0)
    x_out = tf.reshape(x_out, (1, INPUT_SHAPE[1]*2))
    #replaces NaN values with zeros
    x_out = tf.where(tf.math.is_finite(x_out), x_out, tf.zeros_like(x_out))
    return x_out

## TensorFlow Feature Preprocessing Layer

In [154]:
#generating preprocessing layer that will be added to final model
class FeatureGen(tf.keras.layers.Layer):
    #defines custom tensorflow layer 
    def __init__(self):
        #initializes layer
        super(FeatureGen, self).__init__()
    
    def call(self, x_in, MIRROR=False):
        #drop z coordinates if required
        if DROP_Z:
            x_in = x_in[:, :, 0:2]
        if MIRROR:
            #flipping x coordinates
            x_in = np.array(x_in)
            x_in[:, :, 0] = (x_in[:, :, 0]-1)*(-1)
            x_in = tf.convert_to_tensor(x_in)

        #generates list with mean values for landmarks that will be merged
        x_list = [tf.expand_dims(tf_nan_mean(x_in[:, av_set[0]:av_set[0]+av_set[1], :], axis=1), axis=1) for av_set in averaging_sets]
        
        #extracts specific columns from input x_in defined by landmarks
        handedness = right_hand_percentage(x_in)
        if handedness > 0.5:
            x_list.append(tf.gather(x_in, point_landmarks_right, axis=1))
        else: 
            x_list.append(tf.gather(x_in, point_landmarks_left, axis=1))

        #concatenates the two tensors from above along axis 1/columns
        x = tf.concat(x_list, 1)

        #padding to desired length of sequence (defined by LENGTH)
        #get current number of rows
        x_padded = x
        current_rows = tf.shape(x_padded)[0]
        #if current number of rows is greater than desired number of rows, truncate excess rows
        if current_rows > LENGTH:
            x_padded = x_padded[:LENGTH, :, :]

        #if current number of rows is less than desired number of rows, add padding
        elif current_rows < LENGTH:
            #calculate amount of padding needed
            pad_rows = LENGTH - current_rows

            if PADDING ==4: #copy first/last frame
                if pad_rows %2 == 0: #if pad_rows is even
                    padding_front = tf.repeat(x_padded[0:1, :], pad_rows//2, axis=0)
                    padding_back = tf.repeat(x_padded[-1:, :], pad_rows//2, axis=0)
                else: #if pad_rows is odd
                    padding_front = tf.repeat(x_padded[0:1, :], (pad_rows//2)+1, axis=0)
                    padding_back = tf.repeat(x_padded[-1:, :], pad_rows//2, axis=0)
                x_padded = tf.concat([padding_front, x_padded, padding_back], axis=0)
            elif PADDING == 5: #copy last frame
                padding_back = tf.repeat(x_padded[-1:, :], pad_rows, axis=0)
                x_padded = tf.concat([x_padded, padding_back], axis=0)
            else:
                if PADDING ==1: #padding at start and end
                    if pad_rows %2 == 0: #if pad_rows is even
                        paddings = [[pad_rows//2, pad_rows//2], [0, 0], [0, 0]]
                    else: #if pad_rows is odd
                        paddings = [[pad_rows//2+1, pad_rows//2], [0, 0], [0, 0]]
                elif PADDING ==2: #padding only at the end of sequence
                    paddings = [[0, pad_rows], [0, 0], [0, 0]]
                elif PADDING ==3: #no padding
                    paddings = [[0, 0], [0, 0], [0, 0]]
                x_padded = tf.pad(x_padded, paddings, mode='CONSTANT', constant_values=CONSTANT_VALUE)

        x = x_padded
        current_rows = tf.shape(x)[0]

        #interpolate single missing values
        x = pd.DataFrame(np.array(x).flatten()).interpolate(method='linear', limit=2, limit_direction='both')
        #fill missing values with zeros
        x = tf.where(tf.math.is_nan(x), tf.zeros_like(x), x)
        
        #reshape data to 2D or 3D array
        if FLATTEN:
            x = tf.reshape(x, (1, current_rows*INPUT_SHAPE[1]))
        else:
            x = tf.reshape(x, (1, current_rows, INPUT_SHAPE[1]))

        return x

#define converter using generated layer
feature_converter = FeatureGen()

In [155]:
#Tests for generated layer
#One tests symbolic tensor, the other tests real data.
#print(feature_converter(tf.keras.Input((543, 3), dtype=tf.float32, name="inputs")))

#tests preprocessing layer with parquet file
feature_converter((load_relevant_data_subset(f'{LANDMARK_FILES_DIR}{pd.read_csv(TRAIN_FILE).path[10]}')), MIRROR=False)

<tf.Tensor: shape=(1, 22, 129), dtype=float32, numpy=
array([[[ 1.77938476e-01,  7.67818928e-01, -2.48486742e-07, ...,
          5.30708253e-01,  4.24927592e-01, -1.93983670e-02],
        [ 2.06655979e-01,  7.44444609e-01, -1.88931210e-07, ...,
          5.32789052e-01,  4.24572080e-01, -1.91066787e-02],
        [ 2.27484822e-01,  7.34135747e-01, -1.38419338e-07, ...,
          5.34244180e-01,  4.22210097e-01, -2.03100462e-02],
        ...,
        [-1.24098863e-02, -5.25800698e-03,  0.00000000e+00, ...,
          5.32222271e-01,  3.99834335e-01, -2.04068236e-02],
        [-1.32594220e-02, -6.11202093e-03,  0.00000000e+00, ...,
          5.31390190e-01,  3.98376554e-01, -1.74602736e-02],
        [-1.03909485e-02, -3.32162296e-03,  0.00000000e+00, ...,
          5.29494345e-01,  3.99472326e-01, -1.88509431e-02]]],
      dtype=float32)>

In [156]:
def convert_row(i, row, long_sequences, short_sequences, MIRROR=False):

    #loads data from parquet file
    x = load_relevant_data_subset(f'{LANDMARK_FILES_DIR}{row[1].path}')
    
    if MIRROR:
        #applies preprocessing layer to loaded data
        x = feature_converter(tf.convert_to_tensor(x), MIRROR=True).cpu().numpy()

    else:
        #if sequence is too long or too short its index will be added to list, so we can later remove them from dataset
        if x.shape[0] < MIN_LENGTH:
            short_sequences.append(i)
        elif x.shape[0] > MAX_LENGTH:
            long_sequences.append(i)

        #applies preprocessing layer to loaded data
        x = feature_converter(tf.convert_to_tensor(x)).cpu().numpy()


    #returns transformed x data and label of sign
    return x, row[1].label

def convert_and_save_data():
    #reads csv file
    df = pd.read_csv(TRAIN_FILE)
    #maps label number to sign column
    df['label'] = df['sign'].map(label_map)
    
    #filter dataset for participant train/test split
    if CSV_FILTER:      
        #set random state
        random.seed(RANDOM_STATE)
        #get random participants from dataset and add to sample list, number of participants is defined in setup
        sample_list = random.sample(df.groupby('participant_id').mean().index.values.tolist(), TEST_COUNT)
        print(f'These participants are used as test set: {sample_list}.') 
        print(f'Together they have {df.query(f"participant_id in {sample_list}").shape[0]} recordings, {round(df.query(f"participant_id in {sample_list}").shape[0]/df.shape[0]*100,2)}% of the dataset.')
        print('-------------------------------------')
        if TRAIN:
            #limit dataset to participants used for train set
            df = df.query(f'participant_id not in {sample_list}')
        else:
            #limit dataset to participants used for test set
            df = df.query(f'participant_id in {sample_list}')
    if SIGN_FILTER:
        df = df.query(f'label in {sign_list}')
        print(f'These signs are used as dataset: {sign_list}.')
        print(f'Together they have {df.shape[0]} recordings.')
        print('-------------------------------------')

    #sets number of total rows
    total = df.shape[0]
    if MIRROR:
        total = total*2
    #limits number of rows if quick_test is activated
    if QUICK_TEST:
        total = QUICK_LIMIT
    
    #generates numpy array with zeros in shape (total number of rows, number of expected columns)
    if ARRAY: #initialize array with zeros
        if FLATTEN:
            npdata = np.zeros((total, INPUT_SHAPE[0]*INPUT_SHAPE[1]))
        else:
            npdata = np.zeros((total, INPUT_SHAPE[0], INPUT_SHAPE[1]))
        nplabels = np.zeros(total)
    else: #initialize empty array
        if FLATTEN:
            npdata = np.empty((total, INPUT_SHAPE[0]*INPUT_SHAPE[1]))
        else:
            npdata = np.empty((total, INPUT_SHAPE[0], INPUT_SHAPE[1]))
        nplabels = np.empty(total)

    #initializing lists for collecting too long and too short sequences
    long_sequences = []
    short_sequences = []

    #for loop iterates through each row in df dataframe; i is index of the row and row accesses information in the row of df
    #tqdm is used for showing progress bar
    for i, row in tqdm(enumerate(df.iterrows()), total=total/2 if MIRROR else total):
        #load specific parquet file, run preprocessing layer and save x and y data
        (x,y) = convert_row(i, row, long_sequences, short_sequences, MIRROR=False)
        #save x and y to specific row in prepared numpy arrays
        npdata[i,:] = x
        nplabels[i] = y

        if MIRROR:
            (x,y) = convert_row(i, row, long_sequences, short_sequences, MIRROR=True)
            #save x and y to specific row in prepared numpy arrays
            npdata[i+(total//2),:] = x
            nplabels[i+(total//2)] = y

        if QUICK_TEST and i*2 == QUICK_LIMIT - 2:
            break

    if MIRROR:
        #we also need to add too long and too short sequences that were flipped to the lists for removal
        long_sequences.extend([x + (total//2) for x in long_sequences])
        short_sequences.extend([x + (total//2) for x in short_sequences])

    #remove rows of sequences that are too long/too short
    npdata = np.delete(npdata, obj = (long_sequences + short_sequences), axis=0)
    nplabels = np.delete(nplabels, obj = (long_sequences + short_sequences), axis=0)

    print (f'{len(long_sequences)} sequences were removed because they are too long, which is {round(len(long_sequences)/total*100,2)}% of the used dataset.')
    print (f'{len(short_sequences)} sequences were removed because they are too short, which is {round(len(short_sequences)/total*100,2)}% of the used dataset.')
    print('-------------------------------------')
    print('Shapes of the final datasets are:')
    print(npdata.shape, nplabels.shape)

    #save as np file
    np.save(f"{OUTPUT}{feature_data}.npy", npdata)
    np.save(f"{OUTPUT}{feature_labels}.npy", nplabels)
        
convert_and_save_data()

These signs are used as dataset: [31.0, 194.0, 165.0, 234.0, 90.0, 195.0, 239.0, 34.0, 143.0, 182.0, 221.0, 173.0, 55.0, 50.0, 82.0, 52.0, 122.0, 228.0, 155.0, 103.0].
Together they have 7814 recordings.
-------------------------------------


100%|██████████| 7814/7814.0 [00:58<00:00, 133.00it/s]


1020 sequences were removed because they are too long, which is 6.53% of the used dataset.
0 sequences were removed because they are too short, which is 0.0% of the used dataset.
-------------------------------------
Shapes of the final datasets are:
(14608, 22, 129) (14608,)


In [157]:
#test of loading data
X = np.load(f"{OUTPUT}{feature_data}.npy")
y = np.load(f"{OUTPUT}{feature_labels}.npy")

print(X.shape, y.shape)

(14608, 22, 129) (14608,)


## Test for encoding sign labels

In [165]:
#creating new label map with reduced sign number
unique_values = sorted(list(set(y)))  # get a sorted list of unique values
#creating new label map for reduced sign count
new_label_map = {i: unique_values[i] for i in range(len(unique_values))}
#mapping the new label map on y dataset
y_new = np.array([list(new_label_map.keys())[list(new_label_map.values()).index(val)] for val in y])


In [166]:
#creating new dictionary for final new label map used for live prediction at later step
new_dict = {}
for k, v in new_label_map.items():
    new_dict[list(label_map.keys())[int(v)]] = k
print(new_dict)

{'brown': 0, 'callonphone': 1, 'cow': 2, 'cry': 3, 'dad': 4, 'fireman': 5, 'frog': 6, 'gum': 7, 'icecream': 8, 'minemy': 9, 'nose': 10, 'owl': 11, 'please': 12, 'radio': 13, 'shhh': 14, 'shirt': 15, 'tomorrow': 16, 'uncle': 17, 'water': 18, 'who': 19}
